### 1. Definição do problema
> O objetivo deste trabalho é apresentar uma análise sobre os preços de produtos importados pelo Brasil, comparando os valores declarados numa operação de importação com os preços praticados no comércio internacional.
Para isso, primeiramente, será realizado um levantamento de preços junto ao mercado mundial para um grupo de mercadorias de alto valor agregado.
Em seguida, buscar-se-á os preços desses mesmos produtos declarados em processos de importação de mercadorias.
Por fim, far-se-á a comparação entre as médias desses valores, visando a uma conclusão quanto a indícios de fraudes - subfaturamento ou superfaturamento - nas operações de importação realizadas pelas empresas brasileiras.

### 2. A coleta de dados
> Os datasets usados neste trabalho foram coletados de fontes diversas. 
> O DataCoSupplyChainDataset tem como fonte o site __www.kaggle.com__ 

__Bold por favor__ <br>
_Italico Olha isso_ <br>
$ a + bX$ <br>
__________ dfjlkaj dlkjf lkajkdj
> item 
>>item2
>>>item3
- item 1
- item 2
1. item 1
2. item 2 <br>
<font color=blue>"font color=blue"</font><br>
paragrafog <br> outro paragrafo<br>
#isto é comentário

In [19]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

#bibliotecas para requisição de páginas dinâmicas
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
import time 

#Preparando a aquisição das páginas
    #Criando informação sobre quem está fazendo o scrap
headers = {
    'User-Agent': 'João Lima, from PUC-MG University',
    'From': 'jbetol@gmail.com'
}

#Ainda precisa Verificar com Try Except se driver está dispoível
driver = webdriver.Firefox('/media/joao/HDLinux/tools')

#url = "https://www.alibaba.com/trade/search?IndexArea=product_en&CatId=5904002&fsb=y&viewtype=&tab=&SearchScene=&SearchText=phone+display+lcd+screen"

#Vamos pegar 11 páginas de dados e jogar num array
paginas = []
for i in range(1,30):
    url = 'https://www.alibaba.com/products/phone_display_lcd_screen/CID5904002.html?spm=a2700.galleryofferlist.0.0.126c1524tZVgH8&IndexArea=product_en&page='+ str(i)
    paginas.append(url)

print("#####Fim definição de páginas\n\n")
#Dicionário que vai guardar os dados das páginas in memory
desc_items = {} 
for pagina in paginas:

    driver.get(pagina)
    time.sleep(1)
#Verificar modo silencioso para o geckodriver
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser")

    #Aqui é para páginas estáticas. Não usa o webdriver
    #req = requests.get(url)
    #soup = BeautifulSoup(req.text, "html.parser")

    #Adquirindo a lista dos produtos que se encontra na classe indicada
    classes = soup.find('div', class_ = "organic-list app-organic-search__list")

    #Agora vamos buscar os itens da lista
    items =  classes.find_all(class_ = 'organic-gallery-offer-outter J-offer-wrapper')

    #Para cada item, pegar apenas descrição e o preco e guardar num dicionário
    for item in items:
        descricao = item.find('h4').get('title')
        preco = item.find('p', class_ = 'elements-offer-price-normal').get('title')
        desc_items[descricao] = preco 

#transformando o dicionário em DataFrame do Pandas
print("\n#####Fim aquisição de páginas\n\n")

df = pd.DataFrame(desc_items.items(), columns=['Descrição','Preço']) 
df.info()
# displaying the DataFrame  
#print('DataFrame:\n', df) 

# saving the DataFrame as a CSV file 
gfg_csv_data = df.to_csv('lista.csv', header = True) 




#####Fim definição de páginas



#####Fim aquisição de páginas


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Descrição  418 non-null    object
 1   Preço      418 non-null    object
dtypes: object(2)
memory usage: 6.7+ KB
